# Collaborative Filtering-movieLen

movieLen数据集 
- [ml-latest-small(1MB)](https://grouplens.org/datasets/movielens/)
- [ml-latest(234.2MB)](http://files.grouplens.org/datasets/movielens/ml-latest.zip)

In [126]:
import  pandas as pd

movies = pd.read_csv("./data/ml-latest-small/movies.csv")
ratings = pd.read_csv("./data/ml-latest-small/ratings.csv")

# 两个dataframe通过movieId连接

In [127]:
data = pd.merge(movies,ratings,on = 'movieId')
d = data[['userId','rating','movieId','title']].sort_values('userId')
d.to_csv('./data/ml-latest-small/data.csv',index=False)

In [128]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [129]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [130]:
data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [131]:
d.head()

,userId,rating,movieId,title
0,1,4.0,1,Toy Story (1995)
35548,1,4.0,1777,"Wedding Singer, The (1998)"
35249,1,5.0,1732,"Big Lebowski, The (1998)"
34348,1,3.0,1676,Starship Troopers (1997)
2379,1,5.0,50,"Usual Suspects, The (1995)"


In [100]:
file = open("./data/ml-latest-small/data.csv",'r', encoding='UTF-8')
##读取data.csv中每行中除了名字的数据
data = {}##存放每位用户评论的电影和评分
for line in file.readlines():
    line = line.strip().split(',')
    #如果字典中没有某位用户，则使用用户ID来创建这位用户
    if not line[0] in data.keys():
        data[line[0]] = {line[3]:line[1]}
    #否则直接添加以该用户ID为key字典中
    else:
        data[line[0]][line[3]] = line[1]

In [70]:
print(len(data))

611


In [101]:
data['1']

{'Toy Story (1995)': '4.0',
 '"Wedding Singer': '4.0',
 '"Big Lebowski': '5.0',
 'Starship Troopers (1997)': '3.0',
 '"Usual Suspects': '5.0',
 'I Know What You Did Last Summer (1997)': '3.0',
 '"Game': '5.0',
 'Scream 3 (2000)': '5.0',
 'Welcome to Woop-Woop (1997)': '4.0',
 'Kiss the Girls (1997)': '4.0',
 'Conan the Barbarian (1982)': '5.0',
 'Men in Black (a.k.a. MIB) (1997)': '3.0',
 'Face/Off (1997)': '5.0',
 '"Dirty Dozen': '5.0',
 'Con Air (1997)': '4.0',
 'Goldfinger (1964)': '5.0',
 'Austin Powers: International Man of Mystery (1997)': '5.0',
 'L.A. Confidential (1997)': '5.0',
 '"Newton Boys': '5.0',
 'Wild Things (1998)': '4.0',
 'Total Recall (1990)': '4.0',
 '"Honey': '4.0',
 '"Great Mouse Detective': '5.0',
 'Flight of the Navigator (1986)': '4.0',
 '"Black Cauldron': '5.0',
 'Grumpier Old Men (1995)': '4.0',
 'Saving Private Ryan (1998)': '4.0',
 'Bambi (1942)': '5.0',
 "Gulliver's Travels (1939)": '5.0',
 'Back to the Future Part III (1990)': '4.0',
 '"Goonies': '5.0',

# user similarity

## Euclidean

计算两位user之间的相似度。每位user评论的电影不完全一样，利用他们共同评论过的电影计算欧式距离，得到user之间的相似度。

这里返回值越小，相似度越大

In [110]:
from math import *

def Euclidean(user1,user2):
    user1_data=data[user1]
    user2_data=data[user2]
    distance = 0
    for key in user1_data.keys(): # 找到两位用户都评论过的电影，并计算欧式距离
        if key in user2_data.keys():
            distance += pow(float(user1_data[key])-float(user2_data[key]),2) #注意，distance越大表示两者越相似
 
    return 1/(1+sqrt(distance))

In [121]:
# 计算某个用户与其他用户的相似度
def top10_simliar(userID, topn_user):
    res = []
    for userid in data.keys():
        if not userid == userID: # 排除与自己计算相似度
            simliar = Euclidean(userID,userid)
            res.append((userid,simliar))
    res.sort(key=lambda val:val[1])
    return res[:topn_user]

In [122]:
RES = top10_simliar('1',10)
print(RES)

[('68', 0.044330050969940915), ('599', 0.04807925798778345), ('217', 0.04843346156984026), ('160', 0.050181926468153115), ('474', 0.050965218942982136), ('603', 0.05144274016101998), ('608', 0.05197117210837889), ('288', 0.05228876018440262), ('600', 0.053398556747760055), ('313', 0.053908562784313496)]


> 用户之间相似度结果：0表示两位的影评几乎一样，1表示没有共同的影评

## Pearson相关系数

但我们可能会碰到因为两个用户之间数据由于数据膨胀，一方数据大，一方数据小，但是两者称明显的线性关系——分数贬值。

我们引入Pearson相关系数来衡量两个变量之间的线性相关性。范围[-1,1]，-1表示完全负相关，1表示完全正相关，0表示不相关              

-|-
---|---
0.8-1.0 |极强相关
0.6-0.8 |强相关
0.4-0.6 |中等程度相关
0.2-0.4 |弱相关
0.0-0.2 |极弱相关或无相关

In [116]:
def pearson_sim(user1,user2):
    # 取出两位用户评论过的电影和评分
    user1_data = data[user1]
    user2_data = data[user2]
    distance = 0
    common = {}
 
    # 找到两位用户都评论过的电影
    for key in user1_data.keys():
        if key in user2_data.keys():
            common[key] = 1
    if len(common) == 0:
        return 0#如果没有共同评论过的电影，则返回0
    n = len(common)#共同电影数目
    print(n,common)
 
    ##计算评分和
    sum1 = sum([float(user1_data[movie]) for movie in common])
    sum2 = sum([float(user2_data[movie]) for movie in common])
 
    ##计算评分平方和
    sum1Sq = sum([pow(float(user1_data[movie]),2) for movie in common])
    sum2Sq = sum([pow(float(user2_data[movie]),2) for movie in common])
 
    ##计算乘积和
    PSum = sum([float(user1_data[it])*float(user2_data[it]) for it in common])
 
    ##计算相关系数
    num = PSum - (sum1*sum2/n)
    den = sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den == 0:
        return 0
    r = num/den
    return r
 
R = pearson_sim('1','3')
print(R)

7 {'Conan the Barbarian (1982)': 1, 'Bambi (1942)': 1, 'Highlander (1986)': 1, '"Road Warrior': 1, '"Rescuers': 1, 'Tron (1982)': 1, "Schindler's List (1993)": 1}
0.07981885710176334


# UserCF

u2u2i

根据用户推荐电影给其他人

In [124]:
def recommend(user, topn_item, topn_user):
    top_sim_user = top10_simliar(user, topn_user)[0][0]      # 相似度最高的用户
    items = data[top_sim_user]                               # 相似度最高的用户的观影记录
    recommendations = []
    for item in items.keys():                                # 筛选出该用户未观看的电影并添加到列表中
        if item not in data[user].keys():
            recommendations.append((item,items[item]))
    recommendations.sort(key=lambda val:val[1],reverse=True) # 按照评分排序
    return recommendations[:topn_item]                       # 返回评分最高的10部电影
 
Recommendations = recommend('2', 10, 4)
print(Recommendations)

[('Pulp Fiction (1994)', '5.0'), ('Mulholland Drive (2001)', '5.0'), ('Zoolander (2001)', '5.0'), ('Donnie Darko (2001)', '5.0'), ("Rosemary's Baby (1968)", '5.0'), ('Horrible Bosses (2011)', '5.0'), ('Cast Away (2000)', '5.0'), ('Wild Tales (2014)', '5.0'), ('"Clockwork Orange', '5.0'), ('Along Came Polly (2004)', '5.0')]


# ItemCF